<a href="https://colab.research.google.com/github/Arun2005-srm/Deep_Learning_lab/blob/main/lab_4_RA062.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("scolianni/mnistasjpg")

print("Path to dataset files:", path)

100%|██████████| 68.4M/68.4M [00:00<00:00, 211MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/scolianni/mnistasjpg/versions/1


In [ ]:
data = "/root/.cache/kagglehub/datasets/scolianni/mnistasjpg/versions/1"

In [ ]:
import os
os.chdir("/root/.cache/kagglehub/datasets/scolianni/mnistasjpg/versions/1")
os.listdir()

['trainingSample',
 'testSample',
 'testSet.tar.gz',
 'trainingSet',
 'trainingSet.tar.gz',
 'testSet']

In [ ]:
os.chdir("/root/.cache/kagglehub/datasets/scolianni/mnistasjpg/versions/1/trainingSet")
os.listdir()

['trainingSet']

In [ ]:
dataset_root = "/root/.cache/kagglehub/datasets/scolianni/mnistasjpg/versions/1/trainingSet/trainingSet"

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


dataset = datasets.ImageFolder(root=dataset_root, transform=transform)


In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset,[train_size,test_size])

train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = True)


In [ ]:
class_names = dataset.classes
num_classes = len(class_names)
print(f"Classes: {class_names}")

Classes: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
class SimpleFFNN(nn.Module):
    def __init__(self):
        super(SimpleFFNN, self).__init__()
        self.fc1 = nn.Linear(3*28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = SimpleFFNN().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # <-- move data to GPU

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/5], Loss: 0.4460
Epoch [2/5], Loss: 0.2289
Epoch [3/5], Loss: 0.1696
Epoch [4/5], Loss: 0.1348
Epoch [5/5], Loss: 0.1150


In [ ]:
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # <-- move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test images: {100 * correct / total:.2f}%")

Accuracy on test images: 95.89%
